In [ ]:
#!pip install torch torchvision
#!pip install transformers
#!pip install tensorboardx
#!pip install simpletransformers

In [15]:
#from google.colab import drive
#drive.mount('/content/gdrive')

  Using cached pandas-1.1.5-cp36-cp36m-manylinux1_x86_64.whl (9.5 MB)
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
pygaggle 0.0.3.1 requires tokenizers==0.9.4, but you have tokenizers 0.10.3 which is incompatible.
pygaggle 0.0.3.1 requires tqdm==4.45.0, but you have tqdm 4.62.3 which is incompatible.
pygaggle 0.0.3.1 requires transformers==4.0.0, but you have transformers 4.6.1 which is incompatible.
You should consider upgrading via the '/usr/bin/python3 -m pip install --upgrade pip' command.


In [16]:
def load_dataset(file_name):
    import pandas as pd
    import json
    
    df = []
    with open('/mnt/ceph/storage/data-in-progress/data-research/web-search/ECIR-22/ecir22-clickbait-spoiling/' + file_name) as f:
        for i in f:
            i = json.loads(i)
            tweet = i['postText']
            label = i['tags']
            
            assert len(tweet) == 1
            tweet = tweet[0]
            
            assert len(label) == 1
            label = label[0]
            
            assert label in ['phrase', 'passage', 'multi']
            
            if label == 'multi':
                continue
            
            df += [{'text': tweet, 'labels': label == 'phrase'}]

    return df    
            
    
load_dataset('test.jsonl')

AttributeError: module 'pandas' has no attribute 'core'

In [ ]:
from simpletransformers.classification import ClassificationModel
import pandas as pd
import numpy as np
import torch
from scipy.special import softmax
import os
import argparse
from sklearn.metrics import precision_recall_curve
from pathlib import Path
from os import listdir
from os.path import join
from sklearn.model_selection import StratifiedKFold

In [ ]:
args = {
        "overwrite_output_dir": True, 
        "num_train_epochs": 10, 
        "fp16": False, "train_batch_size": 8, 
        "gradient_accumulation_steps": 4, 
        "evaluate_during_training": False, 
        "max_seq_length": 64, 
        "learning_rate": 2e-5, 
        "early_stopping_consider_epochs": True, 
        "output_dir": "outputs/",
        'warmup_ratio': 0.06,
        'warmup_steps': 0,
        'save_steps': 2000
        }

In [ ]:
train_probs = []
train_preds = []
ground_truth = []
ground_questions = []

url="https://github.com/alebondarenko/test/raw/master/data/very_hard.tsv"
df = pd.read_csv(url, error_bad_lines=False, sep='\t')

train_questions = np.array(df.clean.tolist())
y_train = np.array(df['comp'].tolist())

kf = StratifiedKFold(n_splits=10)
for train_index, test_index in kf.split(train_questions, y_train):
    quest_train, quest_test = train_questions[train_index], train_questions[test_index]
    y_tr, y_ts = y_train[train_index], y_train[test_index]
    ground_truth.extend(y_ts)
    ground_questions.extend(quest_test)
    
    train_df = pd.DataFrame({'text': quest_train, 'labels': y_tr})
    tmp_model = ClassificationModel("roberta", "roberta-large", args = args)
    tmp_model.train_model(train_df)
    
    predictions, raw_outputs = tmp_model.predict(quest_test)
    train_preds.extend(predictions)
    train_probs.extend(softmax(raw_outputs, axis=1))

neg_prob = [p[0] for p in train_probs]
pos_prob = [p[1] for p in train_probs]
df_out = pd.DataFrame({'clean': ground_questions,'neg_prob': neg_prob, 'pos_prob': pos_prob, 'predictions': train_preds, 'true_label': ground_truth})


#models = [["roberta", "roberta-large"], ["bert", "bert-large-uncased"], ["albert", "albert-large-v2"]]
#models = [["albert","albert-large-v2"], ["bert","bert-large-uncased"]]
#models = [["electra", "google/electra-large-discriminator"], ["roberta", "roberta-large"], ["xlnet", "xlnet-large-cased"] , ["bert","bert-large-uncased"], ["albert","albert-large-v2"]]
#models = [["electra", "google/electra-base-discriminator"], ["roberta", "roberta-base"], ["xlnet", "xlnet-base-cased"] , ["bert","bert-base-uncased"], ["albert","albert-base-v2"]]